In [ ]:
import openai
import json
from IPython.display import Image, display, Audio, Markdown
import base64

# Preview image for context

# OpenAI API 키 설정
openai.api_key = ''

In [ ]:
def load_data(file_num):
    """
    @description
    주어진 경로에서 JSON 파일을 불러와 오브젝트를 추출하는 함수
    """

    try:
        IMAGE_PATH = f"C:\\workplace\\Kilab\\etri\\dataset\\취합\\평가셋\\images for Task-B\\boxedImage{file_num}.jpg"
        display(Image(IMAGE_PATH))
        with open(IMAGE_PATH, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")
    except :
        base64_image = ""
    
    object_list=[]
    try :
        file_path =f'C:\\workplace\\Kilab\\git\\caption_tool\\typescript\\front\\public\\json\\outputJson\\output_{file_num}.json'
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # 박스를 캡션 수를 기준으로 내림차순
        sorted_boxes = sorted(data['new_bounding_boxes'],key=lambda x: len(x['captions']), reverse=True)
        # 전체 박스 개수
        print("number of box:", len(sorted_boxes))
        # 박스 개수 8개 이하로 줄이기
        if len(sorted_boxes)>8: sorted_boxes = sorted_boxes[:7]
        # print(sorted_boxes)
        
        # 박스 후처리
        # 원래 region 별 캡션 리스트 정보(캡션/region_id)
        # region_descriptions_list = data[list(data.keys())[0]]['region_descriptions_list']
        # # region_id로 매핑할 정보들
        # new_all_regions = data[list(data.keys())[0]]['new_all_regions']
        
        # 캡션 내 키워드 추출
       
        for box in sorted_boxes:
            # object_list에 빈 리스트 추가
            object_list.append([])
            # 캡션 세트 리스트 가져오기
            for captions in box['captions']:
                correct_keywords= [item.lower() for item in captions['caption'].split(' ') if item not in captions['errorCaption'][0].split(' ')]
                [object_list[-1].append(correct_keyword) for correct_keyword in correct_keywords] 
            # break
            # captions = [captions['caption'] for captions in box['captions']]
            # # print("correct captions of box:", captions)
            # for caption in captions:
            #     region_id_list = [region_descriptions['region_id'] for region_descriptions in region_descriptions_list if region_descriptions['phrase'].lower().strip()==caption.lower().strip()]
            #     for region_id in region_id_list:
            #         for region in new_all_regions:
            #             if region['region_id']==region_id:
            #                 # print("3")
            #                 observed_objects = region['synsets']
            #                 if len(observed_objects) > 0 : 
            #                     if isinstance(observed_objects[0], dict):
            #                         observed_objects = [observed_object['synset_name'] for observed_object in observed_objects]
                            
            #                 [object_list[-1].append(observed_object) for observed_object in observed_objects] 
            #                 print
            #                 break

        
    except :
        print("json 데이터 읽는 중 오류 발생")
    finally :
        object_list = [list(set(objects)) for objects in object_list]
        print(object_list)
        return object_list, base64_image
load_data(1160)

In [ ]:
def generate_paragraph(file_num, model="gpt-4o-mini", temperature=0):
    """
    @description
    주어진 입력 데이터에 기반하여 GPT 모델을 통해 추론을 생성하는 함수
    """
    # 오브젝트 리스트 + 이미지 뽑아내기
    object_list, base64_image = load_data(file_num)
    
    # 프롬프트 생성
    # 이미지의 오브젝트간 연관성을 기준으로 큰 범위의 바운딩박스를 생성 
    objects = [ f"box{index}: {objects}" for index, objects in enumerate(object_list)]
    prompt = (
        f"""
        Based on the given image, Create a large range of bounding boxes based on the correlation between regions in the image.
        the large range of bounding boxes according to the following rules and create a paragraph for each large region. 
        Each paragraph should be within 40 words. Provide 2-3 examples as shown below.
        "Be careful not to describe each original region individually."
        
        
        [Region selection rules]

        Perspective
        Object-oriented
        Prioritize highly related objects
        Distinguish unique points of the image
        Adjust prompt complexity based on difficulty
        
        [Objects observed in the image]
        {objects}
            
        """
    )
    
    # GPT 모델에 요청
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a intelligent AI assistant."},
            {"role": "user", "content": [
                {"type":"text","text":prompt},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"
                    }
                }
                ]
             }
        ],
        max_tokens=200,
        n=1,
        # stop=None,
        temperature=temperature
    )
    
    return response.choices[0].message.content.strip()


# 평가 데이터 파일 경로
# 결과 저장을 위한 리스트 초기화

    
def save_json(file_num):
    """
    @description
    generate_paragraph를 호출해 받아온 데이터를 JSON 파일로 저장하는 함수
    """
    
    # generate_paragraph 호출
    # 이미지에 대한 답 생성
    result = ""
    result = generate_paragraph(file_num)
    print(f"예제 ID: {file_num}, GPT 결과: {result}")
    
    # 데이터 생성
    # 기존 output 데이터 + paragraph 키 붙임
    data={}
    try :
        file_path =f'C:\\workplace\\Kilab\\git\\caption_tool\\typescript\\front\\public\\json\\outputJson\\output_{file_num}.json'
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
    except :
        print("json 데이터 읽는 중 오류 발생")
    finally :
        data['paragraph']={'gpt_version':result,
                           'annotator_version':result}
        
        
    with open(f'paragraph{file_num}.json', 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

file_list = [1160, 1161, 1162,1164,1166,1167,1180,1168,1277,1279]
file_num = 2100
[save_json(file_num) for file_num in file_list]